In [1]:
import tensorflow as tf
import os
import cv2
import sys
import dlib
import random 
import numpy as np
import sys
from sklearn.model_selection import train_test_split

F:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
cat_path="cat_pic/"#猫图片的位置
face_path="face_pic/"#人脸图片的位置
imgs=[]#存放图片(None,64,64,3)
labs=[]
#图片的读取和统一shape(64,64,3)
def read_data(path):
    for per in os.listdir(path):
        img_per=cv2.imread(path+per)
        imgs.append(img_per)
        labs.append(path)
    

In [3]:
read_data(cat_path)
read_data(face_path)

In [4]:
#数据的处理和制作标签数据
print("imgs' length is ",len(imgs))
print("labs' length is ",len(labs))
labs=np.array([[0,1] if lab=="cat_pic/" else [1,0] for lab in labs])
imgs=np.array(imgs)
labs=np.array(labs)
print(labs[0])
print(labs[1000])

imgs' length is  2000
labs' length is  2000
[0 1]
[1 0]


In [5]:
train_x,test_x,train_y,test_y=train_test_split(imgs,labs,test_size=0.2)#数据的划分
print("train_x.shape is",train_x.shape)
print("train_y.shape is",train_y.shape)
print("test_x.shape is",test_x.shape)
print("test_y.shape is",test_y.shape)
#数据转化为0～1之间的数求概念需求
train_x=train_x/255.0
test_x=test_x/255.0

train_x.shape is (1600, 64, 64, 3)
train_y.shape is (1600, 2)
test_x.shape is (400, 64, 64, 3)
test_y.shape is (400, 2)


In [6]:
#占位符
x=tf.placeholder(tf.float32,[None,64,64,3],name="x")
y=tf.placeholder(tf.float32,[None,2],name="y")
keep_prob_1=tf.placeholder(tf.float32)
keep_prob_10=tf.placeholder(tf.float32)
print("x",x)
print("y",y)

x Tensor("x:0", shape=(?, 64, 64, 3), dtype=float32)
y Tensor("y:0", shape=(?, 2), dtype=float32)


In [7]:
#定义网络的函数
def weight(shape):
    init=tf.random_normal(shape,stddev=0.1)
    return tf.Variable(init)
def biase(shape):
    init=tf.random_normal(shape,stddev=0.1)
    return tf.Variable(init)
def conv2(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding="SAME")
def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
def dropout(x,keep):
    return tf.nn.dropout(x,keep)

In [8]:
#定义网络
#第一层卷积
w1=weight([4,4,3,32])
b1=biase([32])
wb1=tf.nn.relu(conv2(x,w1)+b1)
pool1=max_pool(wb1)
drop1=dropout(pool1,keep_prob_1)
print("drop1.shape",drop1.shape)
#第二层卷积
w2=weight([3,3,32,64])
b2=biase([64])
wb2=tf.nn.relu(conv2(drop1,w2)+b2)
pool2=max_pool(wb2)
drop2=dropout(pool2,keep_prob_1)
print("drops.shape ",drop2.shape)
W3 = weight([3,3,64,64])
b3 = biase([64])
conv3 = tf.nn.relu(conv2(drop2, W3) + b3)
pool3 = max_pool(conv3)
drop3 = dropout(pool3, keep_prob_1)
#全连接层
Wf = weight([8*16*32, 512])
bf = biase([512])
drop3_flat = tf.reshape(drop3, [-1, 8*16*32])
dense = tf.nn.relu(tf.matmul(drop3_flat, Wf) + bf)
dropf = dropout(dense, keep_prob_10)
print("dropf.shape ",dropf.shape)

#输出层
Wout = weight([512,2])
bout = weight([2])
#out = tf.matmul(dropf, Wout) + bout
out = tf.add(tf.matmul(dropf, Wout), bout)
print("out.shape ",out.shape)
print("x is",x)
print("y is",y)
print("out is",out)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=out,labels=y))
print("cross_entropy.shape ",cross_entropy.shape )
train=tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
accuracy=tf.reduce_mean(tf.cast(tf.equal(tf.argmax(out,1),tf.argmax(y,1)),tf.float32))

drop1.shape (?, 32, 32, 32)
drops.shape  (?, 16, 16, 64)
dropf.shape  (?, 512)
out.shape  (?, 2)
x is Tensor("x:0", shape=(?, 64, 64, 3), dtype=float32)
y is Tensor("y:0", shape=(?, 2), dtype=float32)
out is Tensor("Add:0", shape=(?, 2), dtype=float32)
cross_entropy.shape  ()


In [11]:
batch_size=100
num_batch=train_x.shape[0]#batch_szie
#saver = tf.train.Saver()#模型的保存
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for j in range(10):
        for i in range(num_batch):
            batch_x=train_x[i*batch_size:(i+1)*batch_size]
            batch_y=train_y[i*batch_size:(i+1)*batch_size]
            _,loss=sess.run([train,cross_entropy],
                               feed_dict={x:batch_x,y:batch_y,keep_prob_1:0.5,keep_prob_10:0.75})
            print("the loss is ",loss)
        if (j+1)*batch_size%50==0:
            print("[++++++++++++++++++++++++++++++++++++++++++++++++++]")
            acc = accuracy.eval({x:test_x, y:test_y, keep_prob_1:1.0, keep_prob_10:1.0})
            print("the acc is ",acc)
            print("[++++++++++++++++++++++++++++++++++++++++++++++++++]")
            
    #saver.save(sess, "./model/model.ckpt")#保存模型到当前目录下的 model/model.ckpt  

the loss is  30.99741
the loss is  88.64633
the loss is  37.159946
the loss is  16.144459
the loss is  21.511425
the loss is  29.65139
the loss is  20.997042
the loss is  15.242458
the loss is  7.2472205
the loss is  5.121741
the loss is  6.711995
the loss is  8.18066
the loss is  7.4022427
the loss is  7.5028114
the loss is  5.601233
the loss is  4.197829
[++++++++++++++++++++++++++++++++++++++++++++++++++]
the acc is  0.5325
[++++++++++++++++++++++++++++++++++++++++++++++++++]
the loss is  2.8936918
the loss is  2.31391
the loss is  2.7194746
the loss is  2.2538416
the loss is  2.069006
the loss is  2.9661994
the loss is  2.376247
the loss is  2.8522632
the loss is  1.5604061
the loss is  2.0107315
the loss is  2.0855346
the loss is  1.5518913
the loss is  1.329775
the loss is  1.3423976
the loss is  1.3012908
the loss is  1.4373926
[++++++++++++++++++++++++++++++++++++++++++++++++++]
the acc is  0.575
[++++++++++++++++++++++++++++++++++++++++++++++++++]
the loss is  1.0097322
the lo

In [14]:
if __name__=="__main__":
    saver=tf.train.import_meta_graph("./model/model.ckpt.meta")#加载模型
    graph=tf.get_default_graph()
    x=graph.get_tensor_by_name("x:0")#获取模型中的placeholder括号内为placeholder的名字上面打出来的请参考
    y=graph.get_tensor_by_name("y:0")
    keep1=graph.get_tensor_by_name("Placeholder:0")
    keep10=graph.get_tensor_by_name("Placeholder_1:0")
    prediction=graph.get_tensor_by_name("Add:0")
    img=cv2.imread("101.jpg")#测试的数据
    test_img=cv2.resize(img,(64,64))
    test_img=test_img/255.0
    test_lab=np.array([0,1])
    test_img=np.expand_dims(test_img,axis=0)#扩展一个维度与输入匹配
    test_lab=np.expand_dims(test_lab,axis=0)#扩展一个维度与输入匹配
    print("test_img.shape is ",test_img.shape)
    print("test_lab.shape is",test_lab.shape)
    with tf.Session() as sess:
        saver.restore(sess,"./model/model.ckpt")
        print(sess.run(prediction,feed_dict={x:test_img,y:test_lab,keep1:1.0,keep10:1.0}))
    